# sprint seq2seq

# 【問題1】機械翻訳の実行とコードリーディング
Keras公式のサンプルコードで、短い英語からフランス語への変換を行うものが公開されています。これを動かしてください。


keras/lstm_seq2seq.py at master · keras-team/keras


その上でこのサンプルコードの各部分がどういった役割かを読み取り、まとめてください。以下のようにどこからどこの行が何をしているかを記述してください。

（例）

51から55行目 : ライブラリのimport  
57から62行目 : ハイパーパラメータの設定

In [2]:
from __future__ import print_function

from keras.models import Model
from keras.layers import Input, LSTM, Dense
import numpy as np

batch_size = 64  # Batch size for training.
epochs = 5  # Number of epochs to train for.
latent_dim = 256  # Latent dimensionality of the encoding space.
num_samples = 10000  # Number of samples to train on.
# Path to the data txt file on disk.
data_path = 'fra-eng/fra.txt'

# Vectorize the data.
input_texts = []
target_texts = []
input_characters = set()
target_characters = set()
with open(data_path, 'r', encoding='utf-8') as f:
    lines = f.read().split('\n')
for line in lines[: min(num_samples, len(lines) - 1)]:
    input_text, target_text, _ = line.split('\t')
    # We use "tab" as the "start sequence" character
    # for the targets, and "\n" as "end sequence" character.
    target_text = '\t' + target_text + '\n'
    input_texts.append(input_text)
    target_texts.append(target_text)
    for char in input_text:
        if char not in input_characters:
            input_characters.add(char)
    for char in target_text:
        if char not in target_characters:
            target_characters.add(char)

input_characters = sorted(list(input_characters))
target_characters = sorted(list(target_characters))
num_encoder_tokens = len(input_characters)
num_decoder_tokens = len(target_characters)
max_encoder_seq_length = max([len(txt) for txt in input_texts])
max_decoder_seq_length = max([len(txt) for txt in target_texts])

print('Number of samples:', len(input_texts))
print('Number of unique input tokens:', num_encoder_tokens)
print('Number of unique output tokens:', num_decoder_tokens)
print('Max sequence length for inputs:', max_encoder_seq_length)
print('Max sequence length for outputs:', max_decoder_seq_length)

input_token_index = dict(
    [(char, i) for i, char in enumerate(input_characters)])
target_token_index = dict(
    [(char, i) for i, char in enumerate(target_characters)])

encoder_input_data = np.zeros(
    (len(input_texts), max_encoder_seq_length, num_encoder_tokens),
    dtype='float32')
decoder_input_data = np.zeros(
    (len(input_texts), max_decoder_seq_length, num_decoder_tokens),
    dtype='float32')
decoder_target_data = np.zeros(
    (len(input_texts), max_decoder_seq_length, num_decoder_tokens),
    dtype='float32')

for i, (input_text, target_text) in enumerate(zip(input_texts, target_texts)):
    for t, char in enumerate(input_text):
        encoder_input_data[i, t, input_token_index[char]] = 1.
    encoder_input_data[i, t + 1:, input_token_index[' ']] = 1.
    for t, char in enumerate(target_text):
        # decoder_target_data is ahead of decoder_input_data by one timestep
        decoder_input_data[i, t, target_token_index[char]] = 1.
        if t > 0:
            # decoder_target_data will be ahead by one timestep
            # and will not include the start character.
            decoder_target_data[i, t - 1, target_token_index[char]] = 1.
    decoder_input_data[i, t + 1:, target_token_index[' ']] = 1.
    decoder_target_data[i, t:, target_token_index[' ']] = 1.
# Define an input sequence and process it.
encoder_inputs = Input(shape=(None, num_encoder_tokens))
encoder = LSTM(latent_dim, return_state=True)
encoder_outputs, state_h, state_c = encoder(encoder_inputs)
# We discard `encoder_outputs` and only keep the states.
encoder_states = [state_h, state_c]

# Set up the decoder, using `encoder_states` as initial state.
decoder_inputs = Input(shape=(None, num_decoder_tokens))
# We set up our decoder to return full output sequences,
# and to return internal states as well. We don't use the
# return states in the training model, but we will use them in inference.
decoder_lstm = LSTM(latent_dim, return_sequences=True, return_state=True)
decoder_outputs, _, _ = decoder_lstm(decoder_inputs,
                                     initial_state=encoder_states)
decoder_dense = Dense(num_decoder_tokens, activation='softmax')
decoder_outputs = decoder_dense(decoder_outputs)

# Define the model that will turn
# `encoder_input_data` & `decoder_input_data` into `decoder_target_data`
model = Model([encoder_inputs, decoder_inputs], decoder_outputs)

# Run training
model.compile(optimizer='rmsprop', loss='categorical_crossentropy',
              metrics=['accuracy'])
model.fit([encoder_input_data, decoder_input_data], decoder_target_data,
          batch_size=batch_size,
          epochs=epochs,
          validation_split=0.2)
# Save model
model.save('s2s.h5')

# Next: inference mode (sampling).
# Here's the drill:
# 1) encode input and retrieve initial decoder state
# 2) run one step of decoder with this initial state
# and a "start of sequence" token as target.
# Output will be the next target token
# 3) Repeat with the current target token and current states

# Define sampling models
encoder_model = Model(encoder_inputs, encoder_states)

decoder_state_input_h = Input(shape=(latent_dim,))
decoder_state_input_c = Input(shape=(latent_dim,))
decoder_states_inputs = [decoder_state_input_h, decoder_state_input_c]
decoder_outputs, state_h, state_c = decoder_lstm(
    decoder_inputs, initial_state=decoder_states_inputs)
decoder_states = [state_h, state_c]
decoder_outputs = decoder_dense(decoder_outputs)
decoder_model = Model(
    [decoder_inputs] + decoder_states_inputs,
    [decoder_outputs] + decoder_states)

# Reverse-lookup token index to decode sequences back to
# something readable.
reverse_input_char_index = dict(
    (i, char) for char, i in input_token_index.items())
reverse_target_char_index = dict(
    (i, char) for char, i in target_token_index.items())


def decode_sequence(input_seq):
    # Encode the input as state vectors.
    states_value = encoder_model.predict(input_seq)

    # Generate empty target sequence of length 1.
    target_seq = np.zeros((1, 1, num_decoder_tokens))
    # Populate the first character of target sequence with the start character.
    target_seq[0, 0, target_token_index['\t']] = 1.

    # Sampling loop for a batch of sequences
    # (to simplify, here we assume a batch of size 1).
    stop_condition = False
    decoded_sentence = ''
    while not stop_condition:
        output_tokens, h, c = decoder_model.predict(
            [target_seq] + states_value)

        # Sample a token
        sampled_token_index = np.argmax(output_tokens[0, -1, :])
        sampled_char = reverse_target_char_index[sampled_token_index]
        decoded_sentence += sampled_char

        # Exit condition: either hit max length
        # or find stop character.
        if (sampled_char == '\n' or
           len(decoded_sentence) > max_decoder_seq_length):
            stop_condition = True

        # Update the target sequence (of length 1).
        target_seq = np.zeros((1, 1, num_decoder_tokens))
        target_seq[0, 0, sampled_token_index] = 1.

        # Update states
        states_value = [h, c]

    return decoded_sentence


for seq_index in range(100):
    # Take one sequence (part of the training set)
    # for trying out decoding.
    input_seq = encoder_input_data[seq_index: seq_index + 1]
    decoded_sentence = decode_sequence(input_seq)
    print('-')
    print('Input sentence:', input_texts[seq_index])
    print('Decoded sentence:', decoded_sentence)

Number of samples: 10000
Number of unique input tokens: 71
Number of unique output tokens: 93
Max sequence length for inputs: 16
Max sequence length for outputs: 59
Train on 8000 samples, validate on 2000 samples
Epoch 1/5
8000/8000 [==============================] - 17s 2ms/step - loss: 1.1660 - accuracy: 0.7267 - val_loss: 1.0530 - val_accuracy: 0.7024
Epoch 2/5
8000/8000 [==============================] - 17s 2ms/step - loss: 0.8392 - accuracy: 0.7737 - val_loss: 0.8216 - val_accuracy: 0.7721
Epoch 3/5
8000/8000 [==============================] - 17s 2ms/step - loss: 0.6817 - accuracy: 0.8081 - val_loss: 0.7147 - val_accuracy: 0.7916
Epoch 4/5
8000/8000 [==============================] - 17s 2ms/step - loss: 0.5983 - accuracy: 0.8268 - val_loss: 0.6541 - val_accuracy: 0.8078
Epoch 5/5
8000/8000 [==============================] - 17s 2ms/step - loss: 0.5488 - accuracy: 0.8396 - val_loss: 0.6111 - val_accuracy: 0.8224
-
Input sentence: Go.
Decoded sentence: Prendez nous pas !

-
Input

* 66から69行目：初期化
* 70から84行目：ロードした文章の分割、単語化
* 86から97行目：単語ソート、単語数と文章の最大長さの取得、表示
* 99から102行目：単語の辞書化
* 104から126行目：エンコーダーとデコーダーで使用するために文字化
* 127から143行目：入力シーケンスを定義、エンコーダーとデコーダー設定（LSTM、Dense）
* 145から157行目：学習、コンパイルと保存
* 168から179行目：サンプリングモデルを定義
* 183から186行目：シーケンスをデコードして戻す逆引きトークンインデックス
* 189から224行目：デコード推論の関数
* 227から234行目：推論結果の出力

# 【問題2】イメージキャプショニングの学習済みモデルの実行
上記実装において 5. Test the model の項目を実行してください。また、自身で用意した画像に対しても文章を生成してください。これらに対してどういった文章が出力されたかを記録して提出してください。

データセットからの学習は行わず、学習済みの重みをダウンロードして利用します。

注意点として、デフォルトで設定されている重みのファイル名と、ダウンロードできる重みのファイル名は異なっています。ここは書き換える必要があります。

In [0]:
import os
from PIL import Image
import numpy  as np
import tensorflow as tf
import keras
from tensorflow.keras import backend as K
import matplotlib.pyplot as plt

In [20]:
# 自分のマイドライブにマウントする
from google.colab import drive
drive.mount('/content/drive/')

Drive already mounted at /content/drive/; to attempt to forcibly remount, call drive.mount("/content/drive/", force_remount=True).


In [25]:
# カレントディレクトリの変更
os.chdir('/content/drive/My Drive/sprint23/')
# カレントディレクトリの取得
print(os.getcwd())

/content/drive/My Drive/sprint23


In [0]:
# 1. Clone the repositories
!git clone https://github.com/pdollar/coco.git
!cd coco/PythonAPI/
!make
!python setup.py build
!python setup.py install
!cd ../../
!git clone https://github.com/yunjey/pytorch-tutorial.git

ERROR! Session/line number was not unique in database. History logging moved to new session 59
Cloning into 'coco'...
remote: Enumerating objects: 975, done.
remote: Total 975 (delta 0), reused 0 (delta 0), pack-reused 975
Receiving objects: 100% (975/975), 11.72 MiB | 19.62 MiB/s, done.
Resolving deltas: 100% (575/575), done.
make: *** No targets specified and no makefile found.  Stop.
python3: can't open file 'setup.py': [Errno 2] No such file or directory
python3: can't open file 'setup.py': [Errno 2] No such file or directory
Cloning into 'pytorch-tutorial'...
remote: Enumerating objects: 901, done.
remote: Total 901 (delta 0), reused 0 (delta 0), pack-reused 901
Receiving objects: 100% (901/901), 12.80 MiB | 25.01 MiB/s, done.
Resolving deltas: 100% (482/482), done.


In [26]:
# !cd pytorch-tutorial/tutorials/03-advanced/image_captioning/
os.chdir('./pytorch-tutorial/tutorials/03-advanced/image_captioning/')
# カレントディレクトリの取得
print(os.getcwd())

/content/drive/My Drive/sprint23/pytorch-tutorial/tutorials/03-advanced/image_captioning


In [0]:
# # 2. Download the dataset
# !pip install -r requirements.txt
# !chmod +x download.sh
# !./download.sh

In [0]:
# # 3. Preprocessing
# !python build_vocab.py   
# !python resize.py

In [30]:
# 5. Test the model
!python sample.py --image='png/example.png'

<start> a group of giraffes standing next to each other . <end>


In [32]:
# ホットドッグを食べている人の画像
!python sample.py --image='png/images.jpeg'

<start> a man holding a hot dog in a bun . <end>


# 【問題3】Kerasで動かしたい場合はどうするかを調査
PyTorchによる実装を動かしましたが、何らかの理由からKerasで動かしたい状況が考えられます。どういった手順を踏むことになるか調査し、できるだけ詳しく説明してください。

特に今回はPyTorchのための学習済みの重みをKerasで使えるようにしたいので、その点については必ず触れてください。

学習済みの重みをkerasで使用する場合
* PyTorchの学習済み重みをONNX形式に変換して保存（torch.onnx.exportを使用）
* ONNX形式をkerasで使用できる形式に変換（onnx2keras.onnx_to_kerasを使用）
* チャネルファーストからチャネルラストに変換（PyTorchはチャネルファースト、onnx_to_keraのchange_ordering=True）
* （layerによってはPyTorchからkeras形式に変換できない場合があるため、あらかじめ変換できるようにlayerを調整する）
* kerasで同様のエンコーダー、デコーダーなどをスクラッチする